# Hugging Face REST API
Here you will try out the REST API for invoking the models.

* Use an appropriate model for the task
* Response depends of the task & model in use

**Note:**
An error = 503 indicates that model is in *cold* state but will be loaded. Try again after a few moments.

https://huggingface.co/docs/api-inference/

In [1]:
import sys
import getpass

# setting path for utils package
sys.path.append('../')

## Get the HUGGINGFACEHUB_API_TOKEN

In [2]:
print("Provide the HF API Token:")
HUGGINGFACEHUB_API_TOKEN=getpass.getpass()

Provide the HF API Token:


 ········


## Test models

In [12]:
from utils.hf_post_api import hf_rest_client

hugging_face_model_ids = [
    'tiiuae/falcon-7b-instruct',
    'distilbert/distilbert-base-cased-distilled-squad',
    'mistralai/Mistral-7B-Instruct-v0.2',
    'facebook/bart-large-mnli',
    'google/tapas-base-finetuned-wtq',
    'openlm-research/open_llama_3b_v2',
    'meta-llama/Meta-Llama-3.1-8B-Instruct'
]


## 1. Text generation task

https://huggingface.co/docs/api-inference/detailed_parameters#text-generation-task

In [4]:
# Select the model for the task
# 'mistralai/Mistral-7B-Instruct-v0.2' is a good text gen model
model_index = 2

# Create the client
hf_client = hf_rest_client(hugging_face_model_ids[model_index], api_token=HUGGINGFACEHUB_API_TOKEN)

# Check endpoint URL
print("Model URL: ", hf_client.get_model_url())

Model URL:  https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2


In [5]:
query = "hello friend"
response = hf_client.invoke(query)
response

[{'generated_text': "hello friend,\n\ni'm trying to make a simple game in which the player can move around a 2d world, and interact with objects.\n\ni've been following the tutorials on the wiki, and i've managed to get a player moving around, and i've also managed to make a simple object that the player can interact with.\n\nhowever, i'm having trouble making the object do something when the player interacts with it.\n\n"}]

## 2. Question answering

In [6]:
# Select the model for the task
# 'distilbert/distilbert-base-cased-distilled-squad' is a good Q&A model
model_index = 1

# Create the client
hf_client = hf_rest_client(hugging_face_model_ids[model_index], api_token=HUGGINGFACEHUB_API_TOKEN)

# Check endpoint URL
print("Model URL: ", hf_client.get_model_url())

Model URL:  https://api-inference.huggingface.co/models/distilbert/distilbert-base-cased-distilled-squad


In [7]:
# Q&A tasks expects the input to have an attribute question
query = {
            "question": "What's my name?",
            "context": "My name is Clara and I live in Berkeley.",
}

response = hf_client.invoke(query)
response

{'score': 0.996519148349762, 'start': 11, 'end': 16, 'answer': 'Clara'}

## 3. Zero shot classification
Without any prior examples or fine-tuning, the model needs to classify the sentence into one of the given categories. 
The model attempts to classify the given input into a category. The categories are provided as labels.

In [8]:
# Select the model for the task
# 'facebook/bart-large-mnli' is a good Q&A model
model_index = 3

# Create the client
hf_client = hf_rest_client(hugging_face_model_ids[model_index], api_token=HUGGINGFACEHUB_API_TOKEN)

# Check endpoint URL
print("Model URL: ", hf_client.get_model_url())

Model URL:  https://api-inference.huggingface.co/models/facebook/bart-large-mnli


In [9]:
# Identify movie genre
text = "A retired hitman is forced back into action when a group of assassins targets his family."

# Labels provided in parameters
parameters = {"candidate_labels": ["action", "romance", "comedy"]}

response = hf_client.invoke(text, parameters=parameters)
response

{'sequence': 'A retired hitman is forced back into action when a group of assassins targets his family.',
 'labels': ['action', 'comedy', 'romance'],
 'scores': [0.9827109575271606, 0.012721630744636059, 0.004567424766719341]}

## 4. Table Q&A

In [38]:
# Select the model for the task
# google/tapas-base-finetuned-wtq is a good model for table Q&A

# model_id='google/tapas-large-finetuned-wtq'
model_id='microsoft/tapex-large-finetuned-wtq'

# Create the client
hf_client = hf_rest_client(model_id, api_token=HUGGINGFACEHUB_API_TOKEN)

# Check endpoint URL
print("Model URL: ", hf_client.get_model_url())

Model URL:  https://api-inference.huggingface.co/models/microsoft/tapex-large-finetuned-wtq


In [40]:
table_data = {
  "Product": ["Laptop", "Smartphone", "Tablet", "Headphones", "Laptop", "Smartphone", "Tablet"],
  "Region": ["North America", "Europe", "Asia", "North America", "Europe", "Asia", "North America"],
  "Units Sold": [5000, 8000, 6500, 12000, 4200, 10000, 7000],
  "Revenue (USD)": [5000000, 4800000, 3250000, 1200000, 4200000, 6000000, 3500000],
  "Profit (USD in thousands)": [1200, 800, 650, 300, 1000, 1200, 700]
}

# Sample questions for the data
questions = [
    "Which product category achieved the highest profit in Europe?",
    "How many total units of Smartphones were sold across all regions?",
    "What was the average revenue per unit for Tablets sold in Asia?"
]

# Answers for reference ONLY.
# Models will be used for generating the answers
# answers = [
#     "Laptops achieved the highest profit in Europe, with a profit of $1,000,000.",
#     "A total of 18,000 units of Smartphones were sold across all regions (8,000 in Europe + 10,000 in Asia).",
#     "The average revenue per unit for Tablets sold in Asia was $500 (calculated as $3,250,000 ÷ 6,500 units)."
# ]

question = {
    "query": questions[0],
    "table": table_data
}

response = hf_client.invoke(question)
response

{'answer': ' smartphone'}